## **Data Understanding**

Dataset states terdiri dari 50 baris data (states sebagai unit observasi) dan 12 kolom (variables) dengan detail sebagai beikut :

**state:**  
the full state names (Alabama, Arizona, etc)

**region:**  
region name of the state (North Central, Northeast, South, West)

**abbrv:**  
2-letter abbreviations for the state names (ex: AL for Alabama, AK for Alaska, etc)

**division:**  
division name of the state (middle atlantic, mountain, pacific, etc)

**Population:**  
population estimate

**Income:**  
per capita income

**Illiteracy:**  
the opposite of literacy (percent of population)

**Life Exp:**  
life expectancy in years

**Murder:**  
murder and non-negligent manslaughter rate per 100,000 population

**HS Grad:**  
percent high-school graduates

**Frost:**  
mean number of days with minimum temperature below freezing

**Area:**  
land area in square miles

## **Data Preparation**

In [ ]:
# define url (google drive)
url = 'https://drive.google.com/uc?id={}'.format('1xYI36hZ76zyMECTK0JaSpZdXQDyFRfHx')

# import pandas library
import pandas as pd

# set digit number
pd.set_option('display.float_format', '{:.2f}'.format)

# load file csv from gdrive and save as dataframe
state = pd.read_csv(url)

In [ ]:
# show column info
state.info()

-------------------------------------------------------------------------------
**Comment**

Data memiliki 12 kolom yang terdiri dari 4 data bertipe kategorik yaitu state, region, abbrv, dan division, sedangkan data numerik terdiri dari population, income, illiteracy, life_exp, murder, hs_grad, frost, area

-------------------------------------------------------------------------------

In [ ]:
# show top 5 records
state.head(5)

-------------------------------------------------------------------------------
**Comment**

Unit observasi adalah USA states, dengan kolom state dan abbrv sebagai id observasi, setiap state hanya terasosiasi dengan satu region dan satu division. Kolom lainnya berisi data numerik untuk setiap unit observasi.

-------------------------------------------------------------------------------

## **Exploratory Data Analysis**

### Statistic Summary

In [ ]:
# numerical data
state.describe()

-------------------------------------------------------------------------------
**Comment**

Nilai ekstrim berdasarkan nilai minimum dan maksimum hanya terdapat pada kolom population dan area, sedangkan pada kolom lainnya tidak terdapat nilai ekstrim.

-------------------------------------------------------------------------------

In [ ]:
# categorical data
state.describe(include = 'O')

**Comment**

Region terdiri dari 4 kategori dengan South sebagai region dengan jumlah observasi terbanyak yaitu 16 observasi sedangkan division terbagi menjadi 9 kategori dengan Mountain sebagai division dengan jumlah observasi terbanyak yaitu 8 observasi.

### Correlation Matrix

In [ ]:
# compute correlation
state_corr = state.corr('pearson')

# generate heatmap correlation
state_corr.style.background_gradient(cmap='coolwarm')

-------------------------------------------------------------------------------
**Comment**

Heatmap matriks korelasi menunjukkan bahwa life_exp memiliki korelasi negatif dengan variabel murder (0.78) dan illiteracy (0.59). Sedangkan korelasi positif ditunjukkan oleh variabel hs_grad (0.58).

-------------------------------------------------------------------------------

### Numerical Visualization : Scatterplot

In [ ]:
# generate scatter plot
life_vs_murder = state.plot.scatter('murder', 'life_exp')
life_vs_murder

-------------------------------------------------------------------------------
**Comment**

Terdapat hubungan linear negatif yang kuat antara life_exp vs murder

-------------------------------------------------------------------------------

In [ ]:
# generate scatter plot
life_vs_illiteracy = state.plot.scatter('illiteracy', 'life_exp')
life_vs_illiteracy

-------------------------------------------------------------------------------
**Comment**

Terdapat hubungan linear negatif yang moderate antara life_exp vs illiteracy

-------------------------------------------------------------------------------

In [ ]:
# generate scatter plot
life_vs_hs_grad = state.plot.scatter('hs_grad', 'life_exp')
life_vs_hs_grad

-------------------------------------------------------------------------------
**Comment**

Terdapat hubungan korelasi positif antara life_exp dan hs_grad namun tidak terlalu kuat.

-------------------------------------------------------------------------------

### Categorical Visualization : Boxplot

In [ ]:
# generate boxplot
life_by_region = state.boxplot(column='life_exp', by='region')
life_by_region.set_xticklabels(life_by_region.get_xticklabels(), rotation=45)

-------------------------------------------------------------------------------
**Comment**

Observasi yang berada di region South nampak memiliki karakteristik yang berbeda dibandingkan dengan region lainnya.

-------------------------------------------------------------------------------

In [ ]:
# generate boxplot
life_by_division = state.boxplot(column='life_exp', by='division')
life_by_division.set_xticklabels(life_by_division.get_xticklabels(), rotation=45)

-------------------------------------------------------------------------------
**Comment**

Setiap divisi memiliki distribusi data yang berbeda, namun jika dilihat dari median maka West North Central nampak memiliki karakteristik yang berbeda dibandingkan dengan division lainnya.

-------------------------------------------------------------------------------

## **Data Modelling**

### Hypothesis

-------------------------------------------------------------------------------
semua pengujian hypothesis dilakukan pada level signifikansi 5%

------------------------------------------------------------------------------

Uji-t menguji digunakan untuk menguji signifikansi parameter (koefisien) regresi secara individu

**variabel 1**

H<sub>0</sub>: murder tidak mempengaruhi life_exp (β<sub>1</sub> $=$ 0)

H<sub>1</sub>: murder mempengaruhi life_exp (β<sub>1</sub> $\neq$ 0)

**variabel 2**

H<sub>0</sub>: hs_grad tidak mempengaruhi life_exp (β<sub>2</sub> $=$ 0)

H<sub>1</sub>: hs_grad mempengaruhi life_exp (β<sub>2</sub> $\neq$ 0)

**variabel 3**

H<sub>0</sub>: illiteracy tidak mempengaruhi life_exp (β<sub>3</sub> $=$ 0)

H<sub>1</sub>: illiteracy mempengaruhi life_exp (β<sub>3</sub> $\neq$ 0)

**variabel 4**

H<sub>0</sub>: rata-rata life_exp region South tidak berbeda dengan region lainnya (μ<sub>region_South</sub> $=$ μ<sub>region_Non south</sub>)

H<sub>1</sub>: rata-rata life_exp region South berbeda dengan region lainnya (μ<sub>region_South</sub> $\neq$ μ<sub>region_Non south</sub>)

**variabel 5**

H<sub>0</sub>: rata-rata life_exp division West North Central tidak berbeda dengan division lainnya (μ<sub>division_West north central</sub> $=$ μ<sub>division_Non west north central</sub>)

H<sub>1</sub>: rata-rata life_exp division West North Central berbeda dengan division lainnya (μ<sub>division_West north central</sub> $\neq$ μ<sub>division_Non west north central</sub>)

------------------------------------------------------------------------------

### Regression Modeling

#### model 1: life_exp vs murder, hs_grad, illiteracy

In [ ]:
# import statsmodels library
import statsmodels.api as sm

# variabel input/independen/prediktor
x_1 = sm.add_constant(state[['murder', 'hs_grad', 'illiteracy']])

# variabel output/dependen/target
y = state.life_exp

# fit model
model_1 = sm.OLS(y,x_1).fit()

# print summary
print(model_1.summary())

-------------------------------------------------------------------------------
**Comment**

variabel illiteracy tidak signifikan pada level signifikansi 5% karena p-value = 0.409 > 0.05, sehingga tidak cukup bukti untuk menolak hipotesis nol. Karena illiteracy tidak signifikan, maka illiteracy dapat dikeluarkan dari model sebelum fitting ulang dilakukan.

-------------------------------------------------------------------------------

#### model 2: life_exp vs murder, hs_grad

In [ ]:
# variabel input/independen/prediktor
x_2 = sm.add_constant(state[['murder', 'hs_grad']])

# variabel output/dependen/target
y = state.life_exp

# fit model
model_2 = sm.OLS(y,x_2).fit()

# print summary
print(model_2.summary())

-------------------------------------------------------------------------------
**Comment**

Berdasarkan uji-t, murder dan hs_grad masing-masing berpengaruh terhadap life_exp pada level signifikansi 5% karena p-value **P>|t|** < 0.05, sehingga cukup bukti untuk menolak hipotesis nol.

Berdasarkan uji-F, setidaknya satu dari independent variabel (murder dan hs_grad) berpengaruh terhadap life_exp pada level signifikansi 5% karena p-value **Prob (F-statistic)** = 8.02e-12 < 0.05, sehingga cukup bukti untuk menolak hipotesis nol. Artinya penambahan independent variable (murder dan hs_grad) ke dalam model lebih mampu menjelasakan keragaman dibandingkan model tanpa independent variable.

Nilai Adj. R-squared sebesar 0.648 cukup baik mengindikasikan bahwa 64.8% variabilitas dari life_exp diantara observasi (US States) dapat dijelaskan oleh model.

-------------------------------------------------------------------------------

#### model 3: life_exp vs region_South, division_West North Central

**One-hot Encoding**

One-hot encoding diperlukan dalam pemodelan menggunakan data kategorik. Secara teknis, one-hot encoding memecah satu kolom menjadi beberapa kolom dummy berdasarkan nilai kategorinya. Jumlah kolom yang dihasilkan one-hot encoding adalah sebanyak jumlah kategori. Nilai dari kolom hasil one-hot encoding adalah 1 atau 0, 1 jika observasi (baris data) berasal dari kategori tersebut dan 0 jika bukan.

Sebagai contoh, jika kolom kota berisi kategori 'Jakarta', 'Bogor', Tangsel', maka onehot encoding akan memecah kolom tersebut menjadi 3 kolom yaitu kota_Jakarta, kota_Bogor, dan kota_Tangsel dengan detail nilai kolom sebagai berikut:

*   Jika suatu observasi (baris) memiliki kategori 'Jakarta' maka nilai dari kolom kota_Jakarta pada observasi tersebut akan berisi 1, jika bukan Jakarta maka berisi 0.
*   Jika suatu observasi (baris) memiliki kategori 'Bogor' maka nilai dari kolom kota_Bogor pada observasi tersebut akan berisi 1, jika bukan Bogor maka berisi 0.
*   Jika suatu observasi (baris) memiliki kategori 'Tangsel' maka nilai dari kolom kota_Tangsel pada observasi tersebut akan berisi 1, jika bukan Tangsel maka berisi 0.

In [ ]:
# one hot encoding for categorical variables
state_onehot = pd.get_dummies(state, columns=['region', 'division'], drop_first=False)
state_onehot.head(5)

In [ ]:
# variabel input/independen/prediktor
x_3 = sm.add_constant(state_onehot[['region_South', 'division_West North Central']])

# variabel output/dependen/target
y = state.life_exp

# fit model
model_3 = sm.OLS(y,x_3).fit()

# print summary
print(model_3.summary())

-------------------------------------------------------------------------------
**Comment**

Berdasarkan uji-t, region_South dan division_West North Central masing-masing memiliki rata-rata life_exp yang berbeda dengan region dan division lainnya pada level signifikansi 5% karena p-value **P>|t|** < 0.05, sehingga cukup bukti untuk menolak hipotesis nol.

Berdasarkan uji-F, setidaknya satu dari independent variabel (region_South dan division_West Central North) berpengaruh terhadap life_exp pada level signifikansi 5% karena p-value **Prob (F-statistic)** = 9.58e-07 < 0.05, sehingga cukup bukti untuk menolak hipotesis nol. Artinya penambahan independent variable (region_South dan division_West Central North) ke dalam model lebih mampu menjelasakan keragaman dibandingkan model tanpa independent variable.

Nilai Adj. R-squared sebesar 0.422 mengindikasikan bahwa hanya 42.2% variabilitas dari life_exp diantara observasi (US States) dapat dijelaskan oleh model.

-------------------------------------------------------------------------------

#### model 4: life_exp vs murder, hs_grad, region_South, division_West North Central

In [ ]:
# variabel input/independen/prediktor
x_4 = sm.add_constant(state_onehot[['murder', 'hs_grad', 'region_South', 'division_West North Central']])

# variabel output/dependen/target
y = state.life_exp

# fit model
model_4 = sm.OLS(y,x_4).fit()

# print summary
print(model_4.summary())

-------------------------------------------------------------------------------
**Comment**

Tidak terdapat perbedaan yang nyata antara nilai rata-rata life_exp dari kategorikal variabel (region_South, division_West North Central) karena masing-masing p-value **(P>|t|)** > 0.05 sehingga tidak cukup bukti untuk menolak hipotesis nol.

-------------------------------------------------------------------------------

### Model Evaluation

Berdasarkan nilai-nilai uji-t, uji-F, dan adj. R-squared, maka model yang dipilih sebagai model terbaik dalam analisis ini adalah model 2 (life_exp vs murder, hs_grade) karena beberapa hal berikut


1.   Semua variabel independent (murder, hs_grade) berpengaruh signifikan terhadap variabel response (life_exp) pada level signifikansi 5% berdasarkan uji-t.
2.   Model dengan variabel independent (murder, hs_grade) mampu menjelaskan keragaman life_exp dibanding model tanpa variabel independent pada level signifikansi 5% berdasarkan uji-F.
3. Nilai adj R-squared = 0.648% menggambarkan kemampuan model dalam menjelaskan keragaman life_exp sebesar 64.8%, lebih besar dibandingkan model 3 (life_exp vs region_South, division_West North Central) yang hanya mampu menjelaskan 42,2% keragaman life_exp.





## **Assumptions Checking**

Pemeriksaan asumsi dilakukan terhadap nilai residual dari setiap pengamatan, sehingga perlu terlebih dahulu untuk menghitung nilai residual dengan formula nilai pengamatan - nilai prediksi.

In [ ]:
# import matplotlib library
import matplotlib.pyplot as plt

# import qqplot from statsmodel library
from statsmodels.graphics.gofplots import qqplot

# compute fitted values (y_pred)
fitted_values = model_2.fittedvalues

# compute residuals
residuals = model_2.resid

#### Normalitas

In [ ]:
# generate qqplot
qqplot(residuals, line='s')  # 's' for standardized line
plt.title('Q-Q Plot')
plt.show()

-------------------------------------------------------------------------------
**Comment**

Meskipun tidak semua nilai residual berada pada garis lurus namun sebagian besar nilai residual masih berada pada garis lurus, sehingga dapat disimpulkan memenuhi asumsi kenormalan.

-------------------------------------------------------------------------------

#### Homoskedastisitas

In [ ]:
# generate Residuals vs. Fitted Values
plt.scatter(fitted_values, residuals)
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs. Fitted Values Plot')
plt.axhline(y=0, color='r', linestyle='--')  # Garis horizontal pada y=0
plt.show()


-------------------------------------------------------------------------------
**Comment**

Nilai residuals tersebar secara acak disekitar nilai nol dengan varian konstan mengindikasikan memenuhi asumsi homoskedastisitas.

-------------------------------------------------------------------------------

#### Identifikasi Struktur Non-Acak

In [ ]:
# slice independent variables
murder = state['murder']
hs_grad = state['hs_grad']

# setting canvas layout
plt.figure(figsize=(12, 6))

# generate plot residuals vs Murder
plt.subplot(1, 2, 1)
plt.scatter(murder, residuals)
plt.xlabel('Murder')
plt.ylabel('Residuals')
plt.title('Residuals vs Murder')

# generate plot residuals vs hs_grad
plt.subplot(1, 2, 2)
plt.scatter(hs_grad, residuals)
plt.xlabel('hs_grad')
plt.ylabel('Residuals')
plt.title('Residuals vs HS_grad')

# show plot
plt.tight_layout()
plt.show()

**Comment**

Nilai residuals tersebar secara acak disekitar nilai nol mengindikasikan model regresi mampu menangkap struktur non acak dalam data.

#### Independensi

In [ ]:
import numpy as np

# compute lag 1 from residuals
residuals_lag1 = np.roll(residuals, 1)[1:]

# generate residuals vs residuals_lag1
plt.scatter(residuals_lag1, residuals[1:])
plt.xlabel('Residuals_lag1')
plt.ylabel('Residuals')
plt.title('Residuals vs. Residuals_lag1')
plt.show()

**Comment**

Residuals tidak berkorelasi dengan residuals_lag1 terlihat dari plot yang tidak membentuk pola linear (garis lurus), sehingga memenuhi asumsi independensi residual.

## **Model Interpretation**

Berdasarkan hasil uji-t, uji-F, dan nilai adj R-squared, dapat disimpulkan bahwa model terbaik untuk memodelkan hubungan antara life_exp dengan beberapa variabel independen yang ada dalam data adalah

y = β<sub>0</sub> + β<sub>1</sub>murder + β<sub>2</sub>hs_grad

sesuai dengan output regresi pada model_2, persamaan regresi menjadi

y = 70.3 - 0.24murder + 0.04hs_grad

sehingga dapat diinterpretasikan sebagai berikut:

**intercept**

ketika murder dan hs_grad rates adalah 0%, maka rata-rata life_exp diantara negara-negara bagian (States) di USA diharapkan sebesar 70.3 tahun.

**slope**

Dengan mempertimbangkan nilai variabel hs_grad konstan, maka untuk setiap kenaikan 1% murder rate diharapkan akan menurunkan rata-rata life_exp sebesar 0.24 tahun.

Dengan mempertimbangkan nilai variabel murder konstan, maka untuk setiap kenaikan 1% hs_grad rate diharapkan akan meningkatkan rata-rata life_exp sebesar 0.04 tahun.